Tyfenn Eloy  
Mathias Berthonneau  
Arthur-Louis Bonneau  
Loris Nezan

<h1 style="text-align:center">Process Mining</h1>
<h2 style="text-align:center">Project - Study of potential bias in the rental process</h2>

In [ ]:
import pm4py
import pandas as pd
import matplotlib.pyplot as plt
import warnings

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
log_low = pm4py.read_xes('data/renting_log_low.xes')
log_medium = pm4py.read_xes('data/renting_log_medium.xes')
log_high = pm4py.read_xes('data/renting_log_high.xes')

In [ ]:
log_low.columns

## General trace analysis

In [ ]:
CASE_ID_COL = 'concept:name'
ACTIVITY_COL = 'activity'
TIMESTAMP_COL = 'time:timestamp'

print(f"Case ID Column: {CASE_ID_COL}")
print(f"Activity Column: {ACTIVITY_COL}")
print(f"Timestamp Column: {TIMESTAMP_COL}")

Statistics for the `low` dataset:

In [ ]:
num_cases = len(log_low.groupby(CASE_ID_COL))
num_events = len(log_low)

variants = pm4py.get_variants(log_low)
num_variants = len(variants)

print(f"Number of cases: {num_cases}")
print(f"Number of events: {num_events}")
print(f"Number of variants: {num_variants}")

Statistics for the `medium` dataset:

In [ ]:
num_cases = len(log_medium.groupby(CASE_ID_COL))
num_events = len(log_medium)

variants = pm4py.get_variants(log_medium)
num_variants = len(variants)

print(f"Number of cases: {num_cases}")
print(f"Number of events: {num_events}")
print(f"Number of variants: {num_variants}")

Statistics for the `high` dataset:

In [ ]:
num_cases = len(log_high.groupby(CASE_ID_COL))
num_events = len(log_high)

variants = pm4py.get_variants(log_high)
num_variants = len(variants)

print(f"Number of cases: {num_cases}")
print(f"Number of events: {num_events}")
print(f"Number of variants: {num_variants}")

In [ ]:
#Getting first and last event
first_event = pm4py.get_start_activities(log_low)
last_event = pm4py.get_end_activities(log_low)

print(f"First event: {first_event}")
print(f"Last event: {last_event}")

#Getting first and last event
first_event = pm4py.get_start_activities(log_medium)
last_event = pm4py.get_end_activities(log_medium)

print(f"First event: {first_event}")
print(f"Last event: {last_event}")

#Getting first and last event
first_event = pm4py.get_start_activities(log_high)
last_event = pm4py.get_end_activities(log_high)

print(f"First event: {first_event}")
print(f"Last event: {last_event}")




The general statistics of the different datasets are similar in term of size. We will be able to compare them easily.

## Drawing a general diagram of the flow

In [ ]:
from pm4py.algo.discovery.alpha import algorithm as alpha_miner

In [ ]:
net, initial_marking, final_marking = alpha_miner.apply(log_low)

pm4py.view_petri_net(net, initial_marking, final_marking)

# pm4py.view_case_duration_graph(log_low)

pm4py.view_events_distribution_graph(log_low)

pm4py.discover_directly_follows_graph(log_low)

# pm4py.discover_network_analysis(log_low, out_column=CASE_ID_COL, in_column=CASE_ID_COL, node_column_source='resource', node_column_target='resource', edge_column='concept:name')

In [ ]:
tree = pm4py.discover_prefix_tree(log_low)
# pm4py.view_prefix_tree(tree)

In [ ]:
pm4py.view_footprints(pm4py.discover_footprints(log_low))

In [ ]:
declare = pm4py.discover_declare(log_low)
diag = pm4py.conformance_declare(log_low, declare, return_diagnostics_dataframe=True)

In [ ]:
diag

In the renting process, we can identify 4 main steps:

- The user is applying for a place
- The landlord is accepting or rejecting the application
- The user is paying the rent or not
- The user can get evicted or not

Based on this, we can look for potential bias in the process regarding the additional information we have on the users :

- Age
- Education
- Gender
- Citizenship
- Life situation (marriage, single)

In [ ]:
# split per "case:age"
log_young_low = log_low[log_low['case:age'] < 25]
log_middle_low = log_low[(log_low['case:age'] >= 25) & (log_low['case:age'] <= 50)]
log_old_low = log_low[log_low['case:age'] > 50]

net, initial_marking, final_marking = alpha_miner.apply(log_old_low)
pm4py.view_petri_net(net, initial_marking, final_marking)

In [ ]:
# split per "case:yearsOfEducation"

log_uneducated_low = log_low[log_low['case:yearsOfEducation'] < 5]
log_medium_education_low = log_low[(log_low['case:yearsOfEducation'] >= 5) & (log_low['case:yearsOfEducation'] <= 15)]
log_high_education_low = log_low[log_low['case:yearsOfEducation'] > 16]

net, initial_marking, final_marking = alpha_miner.apply(log_high_education_low)
pm4py.view_petri_net(net, initial_marking, final_marking)

In [ ]:

def printSexism(log, logName):
    
    # split per "case:age"
    log_male = log[log['case:gender'] ==  True]
    log_female = log[log['case:gender'] == False]
    nb_trace_male = len(log_male)
    nb_trace_female = len(log_female)

    # Get len events "Sign Contract" 
    signed_contract_male = len(log_male[log_male['activity'] == 'Sign Contract'])
    signed_contract_female = len(log_female[log_female['activity'] == 'Sign Contract'])

    # Get len events "Reject Prospective Tenant"
    rejected_contract_male = len(log_male[log_male['activity'] == 'Reject Prospective Tenant'])
    rejected_contract_female = len(log_female[log_female['activity'] == 'Reject Prospective Tenant'])

    # get % of rejected contract
    pc_reject_male = nb_trace_male/rejected_contract_male
    pc_reject_female = nb_trace_female/rejected_contract_female

    # get % of accepted contract
    pc_signed_male = nb_trace_male/signed_contract_male
    pc_signed_female = nb_trace_female/signed_contract_female

    #Print the results
    print("-"*30)
    print("Event log: " + logName)
    print("Nombre traces hommes: {}, nombre de signatures: {}, nombre de refus: {}".format(nb_trace_male, signed_contract_male, rejected_contract_male))
    print("-- % refus: {}, % acceptation: {}".format(pc_reject_male, pc_signed_male))
    print("Nombre traces femme: {}, nombre de signatures: {}, nombre de refus: {}".format(nb_trace_female, signed_contract_female, rejected_contract_female))
    print("-- % refus: {}, % acceptation: {}".format(pc_reject_female, pc_signed_female))
    print("-"*30)

In [ ]:
printSexism(log_low, "low")
printSexism(log_medium, "medium")
printSexism(log_high, "high")

In [ ]:
#On cherche a trouver les discrimntion dans les logs
#Cette fois ci on vas regarder les contrats accéptés en fonction de l'age 

def printAgeism(log, logName):
    #split per "case:age"
    log_young = log[log['case:age'] < 25]
    log_middle = log[(log['case:age'] >= 25) & (log['case:age'] <= 50)]
    log_old = log[log['case:age'] > 50]

    # Get len events "Sign Contract" for each age
    signed_contract_young = len(log_young[log_young['activity'] == 'Sign Contract'])
    signed_contract_middle = len(log_middle[log_middle['activity'] == 'Sign Contract'])
    signed_contract_old = len(log_old[log_old['activity'] == 'Sign Contract'])

    # Get len events "Reject Prospective Tenant" for each age
    rejected_contract_young = len(log_young[log_young['activity'] == 'Reject Prospective Tenant'])
    rejected_contract_middle = len(log_middle[log_middle['activity'] == 'Reject Prospective Tenant'])
    rejected_contract_old = len(log_old[log_old['activity'] == 'Reject Prospective Tenant'])

    # get % of rejected contract for each age
    pc_reject_young = rejected_contract_young / len(log_young) * 100
    pc_reject_middle = rejected_contract_middle/len(log_middle) * 100
    pc_reject_old = rejected_contract_old/len(log_old) * 100

    # get % of accepted contract for each age
    pc_signed_young = signed_contract_young/len(log_young) * 100
    pc_signed_middle = signed_contract_middle/len(log_middle) * 100
    pc_signed_old = signed_contract_old/len(log_old) * 100




    #Print the results
    print("-"*30)
    print("Event log: " + logName)
    print("Nombre traces jeunes: {}, nombre de signatures: {}, nombre de refus: {}".format(len(log_young), signed_contract_young, rejected_contract_young))
    print("-- % refus: {}, % acceptation: {}".format(pc_reject_young, pc_signed_young))
    print("Nombre traces moyen: {}, nombre de signatures: {}, nombre de refus: {}".format(len(log_middle), signed_contract_middle, rejected_contract_middle))
    print("-- % refus: {}, % acceptation: {}".format(pc_reject_middle, pc_signed_middle))
    print("Nombre traces vieux: {}, nombre de signatures: {}, nombre de refus: {}".format(len(log_old), signed_contract_old, rejected_contract_old))
    print("-- % refus: {}, % acceptation: {}".format(pc_reject_old, pc_signed_old))

printAgeism(log_low, "low")
printAgeism(log_medium, "medium")
printAgeism(log_high, "high")

In [ ]:
# Ageism regarding eviction 

def AgeismEviction (log, logName):
    #split per "case:age" by tens from 20 to 80
    log_young = log[(log['case:age'] >= 20) & (log['case:age'] <= 30)]
    log_middle = log[(log['case:age'] >= 30) & (log['case:age'] <= 40)]
    log_old = log[(log['case:age'] >= 40) & (log['case:age'] <= 50)]
    log_very_old = log[(log['case:age'] >= 50) & (log['case:age'] <= 60)]
    log_ancient = log[(log['case:age'] >= 60) & (log['case:age'] <= 70)]
    log_very_ancient = log[(log['case:age'] >= 70) & (log['case:age'] <= 80)]
    # Note : les noms de variables sont funny

    # Get len events "Evict Tenant" for each age
    evict_young = len(log_young[log_young['activity'] == 'Evict Tenant'])
    evict_middle = len(log_middle[log_middle['activity'] == 'Evict Tenant'])
    evict_old = len(log_old[log_old['activity'] == 'Evict Tenant'])
    evict_very_old = len(log_very_old[log_very_old['activity'] == 'Evict Tenant'])
    evict_ancient = len(log_ancient[log_ancient['activity'] == 'Evict Tenant']) 
    evict_very_ancient = len(log_very_ancient[log_very_ancient['activity'] == 'Evict Tenant'])

    # # For each age, get the number of cases
    nb_trace_young = len(log_young)
    nb_trace_middle = len(log_middle)
    nb_trace_old = len(log_old)
    nb_trace_very_old = len(log_very_old)
    nb_trace_ancient = len(log_ancient)
    nb_trace_very_ancient = len(log_very_ancient)

    # get % of evicted contract for each age
    pc_evict_young = evict_young / len(log_young) * 100
    pc_evict_middle = evict_middle/len(log_middle) * 100
    pc_evict_old = evict_old/len(log_old) * 100
    pc_evict_very_old = evict_very_old/len(log_very_old) * 100
    pc_evict_ancient = evict_ancient/len(log_ancient) * 100
    pc_evict_very_ancient = evict_very_ancient/len(log_very_ancient) * 100
    

    #Print the results
    print("-"*30)
    print("Event log: " + logName)
    print("Nombre traces jeunes: {}, nombre de evictions: {}".format(nb_trace_young, evict_young))
    print("-- % evictions: {}".format(pc_evict_young))
    print("Nombre traces moyen: {}, nombre de evictions: {}".format(nb_trace_middle, evict_middle))
    print("-- % evictions: {}".format(pc_evict_middle))
    print("Nombre traces vieux: {}, nombre de evictions: {}".format(nb_trace_old, evict_old))
    print("-- % evictions: {}".format(pc_evict_old))
    print("Nombre traces très vieux: {}, nombre de evictions: {}".format(nb_trace_very_old, evict_very_old))
    print("-- % evictions: {}".format(pc_evict_very_old))
    print("Nombre traces ancien: {}, nombre de evictions: {}".format(nb_trace_ancient, evict_ancient))
    print("-- % evictions: {}".format(pc_evict_ancient))
    print("Nombre traces très ancien: {}, nombre de evictions: {}".format(nb_trace_very_ancient, evict_very_ancient))
    print("-- % evictions: {}".format(pc_evict_very_ancient))

AgeismEviction(log_low, "low")
AgeismEviction(log_medium, "medium")
AgeismEviction(log_high, "high")

In [ ]:
#German speaking racism regarding eviction

def GermanismEviction (log, logName):
    log_geman = log[log['case:german speaking'] == True]
    log_not_german = log[log['case:german speaking'] == False]

    # Get len events "Evict Tenant" for each language
    evict_german = len(log_geman[log_geman['activity'] == 'Evict Tenant'])
    evict_not_german = len(log_not_german[log_not_german['activity'] == 'Evict Tenant'])

    # # For each language, get the number of cases
    nb_trace_german = len(log_geman)
    nb_trace_not_german = len(log_not_german)

    # get % of evicted contract for each language
    pc_evict_german = evict_german / len(log_geman) * 100
    pc_evict_not_german = evict_not_german/len(log_not_german) * 100

    #Print the results
    print("-"*30)
    print("Event log: " + logName)
    print("Nombre traces german: {}, nombre de evictions: {}".format(nb_trace_german, evict_german))
    print("-- % evictions: {}".format(pc_evict_german))
    print("Nombre traces not german: {}, nombre de evictions: {}".format(nb_trace_not_german, evict_not_german))
    print("-- % evictions: {}".format(pc_evict_not_german))
    

In [ ]:
GermanismEviction(log_low, "low")
GermanismEviction(log_medium, "medium")
GermanismEviction(log_high, "high")


In [ ]:
def getGermanAcceptance(log, logname):
    log_german = log[log['case:german speaking'] == True]
    log_not_german = log[log['case:german speaking'] == False]

    # Get len events "Sign Contract" for each language
    signed_contract_german = len(log_german[log_german['activity'] == 'Sign Contract'])
    signed_contract_not_german = len(log_not_german[log_not_german['activity'] == 'Sign Contract'])

    # Get len events "Reject Prospective Tenant" for each language
    rejected_contract_german = len(log_german[log_german['activity'] == 'Reject Prospective Tenant'])
    rejected_contract_not_german = len(log_not_german[log_not_german['activity'] == 'Reject Prospective Tenant'])

    # get % of rejected contract for each language
    pc_reject_german = rejected_contract_german / len(log_german) * 100
    pc_reject_not_german = rejected_contract_not_german/len(log_not_german) * 100

    # get % of accepted contract for each language
    pc_signed_german = signed_contract_german/len(log_german) * 100
    pc_signed_not_german = signed_contract_not_german/len(log_not_german) * 100

    #Print the results
    print("-"*30)
    print("Event log: " + logname)
    print("Nombre traces german: {}, nombre de signatures: {}, nombre de refus: {}".format(len(log_german), signed_contract_german, rejected_contract_german))
    print("-- % refus: {}, % acceptation: {}".format(pc_reject_german, pc_signed_german))
    print("Nombre traces not german: {}, nombre de signatures: {}, nombre de refus: {}".format(len(log_not_german), signed_contract_not_german, rejected_contract_not_german))
    print("-- % refus: {}, % acceptation: {}".format(pc_reject_not_german, pc_signed_not_german))
    print("-"*30)

In [ ]:
getGermanAcceptance(log_low, "low")
getGermanAcceptance(log_medium, "medium")
getGermanAcceptance(log_high, "high")

In [ ]:
def getCitizenAcceptance(log,logname):
    log_citizen = log[log['case:citizen'] == True]
    log_not_citizen = log[log['case:citizen'] == False]

    signed_contract_citizen = len(log_citizen[log_citizen['activity'] == 'Sign Contract'])
    signed_contract_not_citizen = len(log_not_citizen[log_not_citizen['activity'] == 'Sign Contract'])

    rejected_contract_citizen = len(log_citizen[log_citizen['activity'] == 'Reject Prospective Tenant'])
    rejected_contract_not_citizen = len(log_not_citizen[log_not_citizen['activity'] == 'Reject Prospective Tenant'])

    pc_reject_citizen = rejected_contract_citizen / len(log_citizen) * 100
    pc_reject_not_citizen = rejected_contract_not_citizen/len(log_not_citizen) * 100

    pc_signed_citizen = signed_contract_citizen/len(log_citizen) * 100
    pc_signed_not_citizen = signed_contract_not_citizen/len(log_not_citizen) * 100

    #Print the results
    print("-"*30)
    print("Event log: " + logname)
    print("Nombre traces citizen: {}, nombre de signatures: {}, nombre de refus: {}".format(len(log_citizen), signed_contract_citizen, rejected_contract_citizen))
    print("-- % refus: {}, % acceptation: {}".format(pc_reject_citizen, pc_signed_citizen))
    print("Nombre traces not citizen: {}, nombre de signatures: {}, nombre de refus: {}".format(len(log_not_citizen), signed_contract_not_citizen, rejected_contract_not_citizen))
    print("-- % refus: {}, % acceptation: {}".format(pc_reject_not_citizen, pc_signed_not_citizen))
    print("-"*30)

In [ ]:
getCitizenAcceptance(log_low, "low")
getCitizenAcceptance(log_medium, "medium")
getCitizenAcceptance(log_high, "high")

In [ ]:
# racism by marital status
def getMarriedAcceptance(log, logname): 
    log_married = log[log['case:married'] == True]
    log_not_married = log[log['case:married'] == False]

    signed_contract_married = len(log_married[log_married['activity'] == 'Sign Contract'])
    signed_contract_not_married = len(log_not_married[log_not_married['activity'] == 'Sign Contract'])

    rejected_contract_married = len(log_married[log_married['activity'] == 'Reject Prospective Tenant'])
    rejected_contract_not_married = len(log_not_married[log_not_married['activity'] == 'Reject Prospective Tenant'])

    pc_reject_married = rejected_contract_married / len(log_married) * 100
    pc_reject_not_married = rejected_contract_not_married/len(log_not_married) * 100

    pc_signed_married = signed_contract_married/len(log_married) * 100
    pc_signed_not_married = signed_contract_not_married/len(log_not_married) * 100

    #Print the results
    print("-"*30)
    print("Event log: " + logname)
    print("Nombre traces married: {}, nombre de signatures: {}, nombre de refus: {}".format(len(log_married), signed_contract_married, rejected_contract_married))
    print("-- % refus: {}, % acceptation: {}".format(pc_reject_married, pc_signed_married))
    print("Nombre traces not married: {}, nombre de signatures: {}, nombre de refus: {}".format(len(log_not_married), signed_contract_not_married, rejected_contract_not_married))
    print("-- % refus: {}, % acceptation: {}".format(pc_reject_not_married, pc_signed_not_married))
    print("-"*30)

In [ ]:
getMarriedAcceptance(log_low, "low")
getMarriedAcceptance(log_medium, "medium")
getMarriedAcceptance(log_high, "high")


## Eviction analysis

### Eviction rate

In [ ]:
# get eviction
def get_number_of_evictions(log, logname):
    log_evict = log[log['activity'] == 'Evict Tenant']
    percentage = len(log_evict) / len(log) * 100
    print("Event log: " + logname)
    print("Nombre traces evictions: {}".format(len(log_evict)))
    print("- % evictions: {}".format(percentage))


In [ ]:
get_number_of_evictions(log_low, "low")
get_number_of_evictions(log_medium, "medium")
get_number_of_evictions(log_high, "high")



In [ ]:
# get number of average issued warnings per case where eviction is issued
def get_average_number_of_warnings(log, logname):
    log_evict = log[log['activity'] == 'Evict Tenant']
    log_issue_warning = log[log['activity'] == 'Issue Warning']
    warnings = log_issue_warning.groupby('case:concept:name').size()
    average = warnings.mean()
    print("Event log: " + logname)
    print("Nombre moyen de warnings par case: {}".format(average))
    

In [ ]:
get_average_number_of_warnings(log_low, "low")
get_average_number_of_warnings(log_medium, "medium")
get_average_number_of_warnings(log_high, "high")


In [ ]:
# get number of average issued warnings per case where eviction is issued per age


In [ ]:
def getWarningPerAge():
    logs = pd.concat([log_low, log_medium, log_high])

    # Filter cases where eviction is issued
    eviction_cases = logs[logs['activity'] == 'Move In']    
    
    # Calculate number of warnings issued per case
    warnings_per_case = logs[logs['activity'] == 'Reject Prospective Tenant'].groupby('case:concept:name').size().rename('num_warnings')
    warnings_per_case = warnings_per_case.fillna(0)
    
    # Merge eviction cases with warnings issued
    eviction_cases_with_warnings = eviction_cases.merge(warnings_per_case, on='case:concept:name', how='left')
    eviction_cases_with_warnings['num_warnings'] = eviction_cases_with_warnings['num_warnings'].fillna(0)
    
    # Calculate average warnings per case where eviction is issued per age group
    average_warnings_per_age = eviction_cases_with_warnings.groupby('case:age')['num_warnings'].sum()
    
    print("all logs included", "\n", average_warnings_per_age)

getWarningPerAge()

def getAccepPerYearOfEducation(log, log_type):
    
    nb_accepted = len(log[log['activity'] == 'Sign Contract'])
    nb_rejected = len(log[log['activity'] == 'Reject Prospective Tenant'])
    nb_years_of_education = len(log.groupby('case:yearsOfEducation'))
    
    nb_accepted_per_year = log[log['activity'] == 'Sign Contract'].groupby('case:yearsOfEducation').size()
    if len(nb_accepted_per_year) < nb_years_of_education:
        for i in range(nb_years_of_education):
            if i not in nb_accepted_per_year.index:
                nb_accepted_per_year[i] = 0
    nb_rejected_per_year = log[log['activity'] == 'Reject Prospective Tenant'].groupby('case:yearsOfEducation').size()
    if len(nb_rejected_per_year) < nb_years_of_education:
        for i in range(nb_years_of_education):
            if i not in nb_rejected_per_year.index:
                nb_rejected_per_year[i] = 0
    
    nb_per_year = nb_rejected_per_year.add(nb_accepted_per_year, fill_value=0)
    print(nb_per_year)
    
    percentage_total_evicted_per_year = nb_accepted_per_year / nb_accepted * 100
    percentage_total_rejected_per_year = nb_rejected_per_year / nb_rejected * 100
    
    percentage_accepted_per_year = (nb_accepted_per_year / nb_per_year * 100).fillna(0)
    percentage_rejected_per_year = (nb_rejected_per_year / nb_per_year * 100).fillna(0)
    
    
    print("Event log: " + log_type)
    print("Nombre de contrat acceptés par année d'étude: {}".format(nb_accepted_per_year))
    print("Nombre de contrat refusés par année d'étude: {}".format(nb_rejected_per_year))
    print("Poucentage total de contrat acceptés par année d'étude (dans le groupe) : {}".format(percentage_total_accepted_per_year))
    print("Poucentage total de contrat refusés par année d'étude (dans le groupe) : {}".format(percentage_total_rejected_per_year))
    
    print("Pourcentage de contrat acceptés par année d'étude selon les cas acceptés ou refusés: {}".format(percentage_accepted_per_year))
    print("Pourcentage de contrat refusés par année d'étude selon les cas acceptés ou refusés: {}".format(percentage_rejected_per_year))
    
    return nb_accepted_per_year, nb_rejected_per_year, percentage_total_accepted_per_year, percentage_total_rejected_per_year, percentage_accepted_per_year, percentage_rejected_per_year

def getEvictionPerYearOfEducation(log, log_type):
    
    nb_evicted = len(log[log['activity'] == 'Evict Tenant'])
    nb_not_evicted = len(log[log['activity'] == 'Move In'])
    nb_years_of_education = len(log.groupby('case:yearsOfEducation'))
    
    nb_evicted_per_year = log[log['activity'] == 'Evict Tenant'].groupby('case:yearsOfEducation').size()
    if len(nb_evicted_per_year) < nb_years_of_education:
        for i in range(nb_years_of_education):
            if i not in nb_evicted_per_year.index:
                nb_evicted_per_year[i] = 0
    nb_not_evicted_per_year = log[log['activity'] == 'Move In'].groupby('case:yearsOfEducation').size()
    if len(nb_not_evicted_per_year) < nb_years_of_education:
        for i in range(nb_years_of_education):
            if i not in nb_not_evicted_per_year.index:
                nb_not_evicted_per_year[i] = 0
    
    nb_per_year = nb_evicted_per_year.add(nb_not_evicted_per_year, fill_value=0)
    print(nb_per_year)
    
    percentage_total_evicted_per_year = nb_evicted_per_year / nb_evicted * 100
    percentage_total_not_evicted_per_year = nb_not_evicted_per_year / nb_not_evicted * 100
    
    percentage_evicted_per_year = (nb_evicted_per_year / nb_per_year * 100).fillna(0)
    percentage_not_evicted_per_year = (nb_not_evicted_per_year / nb_per_year * 100).fillna(0)
    
    
    print("Event log: " + log_type)
    print("Nombre de contrat evicted par année d'étude: {}".format(nb_evicted_per_year))
    print("Poucentage total de contrat evicted par année d'étude (dans le groupe) : {}".format(percentage_total_evicted_per_year))
    print("Pourcentage de contrat evicted par année d'étude selon les cas evicted ou not evicted: {}".format(percentage_evicted_per_year))
    
    return nb_evicted_per_year, nb_not_evicted_per_year, percentage_total_evicted_per_year, percentage_total_not_evicted_per_year, percentage_evicted_per_year, percentage_not_evicted_per_year

%matplotlib inline
# for log in [log_low, log_medium, log_high]:
#     log_type = 'low' if log is log_low else 'medium' if log is log_medium else 'high'
#     print("log type: ", log_type)
#     nb_accepted_per_year, nb_rejected_per_year, percentage_total_accepted_per_year, percentage_total_rejected_per_year, percentage_accepted_per_year, percentage_rejected_per_year = getAccepPerYearOfEducation(log, log_type)
#     
#     # print as a bar chart
#     fig, ax = plt.subplots()
#     ax.bar(nb_rejected_per_year.index, nb_rejected_per_year.values, label='Rejected', color='red')
#     ax.bar(nb_accepted_per_year.index, nb_accepted_per_year.values, label='Accepted', color='green')
#     ax.set_xlabel('Years of education')
#     ax.set_ylabel('Number of cases')
#     ax.set_title('Number of cases per years of education')
#     ax.legend()
#     plt.show()
#     
#     # print as a bar chart with %centage of accepted/rejected, with 100% at the top, and 0% at the bottom, with rejected on top of accepted
#     for rejected_idx, rejected_year in enumerate(nb_rejected_per_year.index):
#         print(rejected_idx+1, rejected_year)
#         if not rejected_year in nb_accepted_per_year.index:
#             nb_accepted_per_year[rejected_year] = 0    
#     for accepted_idx, accepted_year in enumerate(nb_accepted_per_year.index):
#         if not accepted_year in nb_rejected_per_year.index:
#             nb_rejected_per_year[accepted_year] = 0
#     nb_accepted_per_year = nb_accepted_per_year.sort_index()
#     nb_rejected_per_year = nb_rejected_per_year.sort_index()
#     
#     fig, ax = plt.subplots()
#     ax.bar(nb_rejected_per_year.index, percentage_rejected_per_year.values, label='Rejected', color='red', bottom=percentage_accepted_per_year.values)
#     ax.bar(nb_accepted_per_year.index, percentage_accepted_per_year.values, label='Accepted', color='green')
#     ax.set_xlabel('Years of education')
#     ax.set_ylabel('Percentage of cases')
#     ax.set_title('Percentage of cases per years of education')
#     ax.legend()

for log in [log_low, log_medium, log_high]:
    log_type = 'low' if log is log_low else 'medium' if log is log_medium else 'high'
    print("log type: ", log_type)
    nb_evicted_per_year, nb_not_evicted_per_year, percentage_total_evicted_per_year, percentage_total_rejected_per_year, percentage_accepted_per_year, percentage_rejected_per_year = getEvictionPerYearOfEducation(log, log_type)

    # print as a bar chart
    fig, ax = plt.subplots()
    ax.bar(nb_not_evicted_per_year.index, nb_not_evicted_per_year.values, label='Not evicted', color='Yellow')
    ax.bar(nb_evicted_per_year.index, nb_evicted_per_year.values, label='Evicted', color='red')
    ax.set_xlabel('Years of education')
    ax.set_ylabel('Number of cases')
    ax.set_title('Number of cases per years of education')
    ax.legend()
    plt.show()

    # print as a bar chart with %centage of accepted/rejected, with 100% at the top, and 0% at the bottom, with rejected on top of accepted
    for rejected_idx, rejected_year in enumerate(nb_not_evicted_per_year.index):
        print(rejected_idx+1, rejected_year)
        if not rejected_year in nb_evicted_per_year.index:
            nb_evicted_per_year[rejected_year] = 0    
    for accepted_idx, accepted_year in enumerate(nb_evicted_per_year.index):
        if not accepted_year in nb_not_evicted_per_year.index:
            nb_not_evicted_per_year[accepted_year] = 0
    nb_evicted_per_year = nb_evicted_per_year.sort_index()
    nb_not_evicted_per_year = nb_not_evicted_per_year.sort_index()

    fig, ax = plt.subplots()
    ax.bar(nb_not_evicted_per_year.index, percentage_rejected_per_year.values, label='Not evicted', color='yellow', bottom=percentage_accepted_per_year.values)
    ax.bar(nb_evicted_per_year.index, percentage_accepted_per_year.values, label='Evicted', color='red')
    ax.set_xlabel('Years of education')
    ax.set_ylabel('Percentage of cases')
    ax.set_title('Percentage of cases per years of education')
    ax.legend()

In [ ]:
from pm4py.algo.filtering.log.variants import variants_filter
from pm4py.algo.discovery.heuristics import algorithm as heuristiics_miner
from pm4py.algo.evaluation.replay_fitness import algorithm as replay_fitness_factory
from pm4py.algo.discovery.dfg import algorithm as dfg_factory

import pandas as pd

# Subset selection - selecting more frequent variants
variants_low = variants_filter.get_variants(log_low)
variants_medium = variants_filter.get_variants(log_medium)
variants_high = variants_filter.get_variants(log_high)

# Process Discovery using Alpha Miner
net, initial_marking, final_marking = alpha_miner.apply(log_low)
pm4py.view_petri_net(net, initial_marking, final_marking)


for log in [log_low, log_medium, log_high]:
    log_type = 'low' if log is log_low else 'medium' if log is log_medium else 'high'
    variants = variants_filter.get_variants(log)
    print("log type: ", log_type)
    print("number of variants: ", variants)


    map_heu = heuristiics_miner.apply_heu(log)
    net_heu, initial_marking_heu, final_marking_heu = heuristiics_miner.apply(log)
    pm4py.view_heuristics_net(map_heu)
    pm4py.view_petri_net(net_heu, initial_marking_heu, final_marking_heu)

    # Quality evaluation using Replay Fitness
    replay_fitness_alpha = replay_fitness_factory.apply(log_low, net, initial_marking, final_marking)
    replay_fitness_heu = replay_fitness_factory.apply(log_low, net_heu, initial_marking_heu, final_marking_heu)

    print("replay fitness alpha: ", replay_fitness_alpha)
    print("replay fitness heu: ", replay_fitness_heu)

    # DFG
    dfg_log = dfg_factory.apply(log)
    start_activities = pm4py.get_start_activities(log)
    end_activities = pm4py.get_end_activities(log)
    pm4py.view_dfg(dfg_log, start_activities, end_activities)
    
    # Fuzzy Miner Analysis
    

In [ ]:
#Clustering with decision point mining
from sklearn.cluster import AgglomerativeClustering
import pandas as pd
import numpy as np

df = log_high

variants = variants_filter.get_variants(df)
variants_dict = {v: i for i, v in enumerate(variants)}
variants_list = [v for v in variants]


# Define a function to compute Levenshtein distance
def levenshtein_distance(s1, s2):
    if len(s1) < len(s2):
        return levenshtein_distance(s2, s1)
    if len(s2) == 0:
        return len(s1)
    
    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

# Compute the Levenshtein distance matrix
distance_matrix = np.zeros((len(variants_dict), len(variants_dict)))
for i, s1 in enumerate(variants_dict):
    for j, s2 in enumerate(variants_dict):
        distance_matrix[i][j] = levenshtein_distance(s1, s2)

# Apply agglomerative clustering
num_clusters = 2
clustering = AgglomerativeClustering(n_clusters=num_clusters, linkage='complete')
cluster_assignments = clustering.fit_predict(distance_matrix)

# Silhouette coefficient
from sklearn.metrics import silhouette_score
print("Silhouette coefficient: {}".format(silhouette_score(distance_matrix, cluster_assignments)))

#produire le dendrogramme
from scipy.cluster.hierarchy import dendrogram
import matplotlib.pyplot as plt
from scipy.cluster import hierarchy

# Calcul de la matrice de liaison
linkage_matrix = hierarchy.complete(distance_matrix)

# Affichage du dendrogramme
plt.figure(figsize=(10, 7))
dn = hierarchy.dendrogram(linkage_matrix, leaf_rotation=90)
plt.title("Dendrogramme du clustering hiérarchique")
plt.xlabel('Variants')
plt.ylabel('Distance')
plt.show()

